coggers is my favorite song from interstellar, let's see what it looks like

In [1]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
sample_rate, coggers = wav.read('data/hans_zimmer/coggers.wav')

In [3]:
length = len(coggers)

length, sample_rate

(10590336, 44100)

In [4]:
# song length: 240 seconds

length/240 # approximately the sample rate

44126.4

In [5]:
# split the data into two tracks for separate training

a = coggers[:,0]
b = coggers[:,1]

In [6]:
# plt.plot(a, 'b')
# plt.plot(b, 'r')

In [7]:
# HYPERPARAMETERS

SEQ_LEN = 0.5 # this is in seconds
SEQ_LEN = int(SEQ_LEN * sample_rate)
print(SEQ_LEN)

VAL_PCT = 0.2

# gonna list some powers of 2 here for reference
# 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144, 524288, 1048576, 2097152
HIDDEN_DIM = 1024
N_LAYERS = 2
OUT_DIM = 1

EPOCHS = 4
LR = 0.001

22050


In [8]:
scaler_a = MinMaxScaler(feature_range=(-1,1))
scaler_b = MinMaxScaler(feature_range=(-1,1))

a = scaler_a.fit_transform(a.reshape(-1,1))
b = scaler_b.fit_transform(b.reshape(-1,1))

In [9]:
a = torch.tensor(a).cuda().float()
b = torch.tensor(b).cuda().float()

In [10]:
'''data_a = []
data_b = []

for i in tqdm(range(len(a)-SEQ_LEN)):
    data_a.append(a[i:i+SEQ_LEN])
    data_b.append(b[i:i+SEQ_LEN])

X_a = [i[:-1] for i in data_a]
Y_a = [i[-1] for i in data_a]

X_b = [i[:-1] for i in data_b]
Y_b = [i[-1] for i in data_b]'''

'data_a = []\ndata_b = []\n\nfor i in tqdm(range(len(a)-SEQ_LEN)):\n    data_a.append(a[i:i+SEQ_LEN])\n    data_b.append(b[i:i+SEQ_LEN])\n\nX_a = [i[:-1] for i in data_a]\nY_a = [i[-1] for i in data_a]\n\nX_b = [i[:-1] for i in data_b]\nY_b = [i[-1] for i in data_b]'

In [11]:
'''X_a = np.array(X_a, dtype='float32').squeeze()
X_b = np.array(X_b).squeeze()
Y_a = np.array(Y_a).squeeze()
Y_b = np.array(Y_b).squeeze()

X_a = torch.tensor(X_a).float().cuda()
Y_a = torch.tensor(Y_a).float().cuda()
X_b = torch.tensor(X_b).float().cuda()
Y_a = torch.tensor(Y_a).float().cuda()'''

"X_a = np.array(X_a, dtype='float32').squeeze()\nX_b = np.array(X_b).squeeze()\nY_a = np.array(Y_a).squeeze()\nY_b = np.array(Y_b).squeeze()\n\nX_a = torch.tensor(X_a).float().cuda()\nY_a = torch.tensor(Y_a).float().cuda()\nX_b = torch.tensor(X_b).float().cuda()\nY_a = torch.tensor(Y_a).float().cuda()"

In [12]:
'''train_set_a = TensorDataset(X_a, Y_a)
train_set_b = TensorDataset(X_b, Y_b)

loader_a = DataLoader(train_set_a, batch_size=BATCH_SIZE, shuffle=False)
loader_b = DataLoader(train_set_b, batch_size=BATCH_SIZE, shuffle=False)'''

'train_set_a = TensorDataset(X_a, Y_a)\ntrain_set_b = TensorDataset(X_b, Y_b)\n\nloader_a = DataLoader(train_set_a, batch_size=BATCH_SIZE, shuffle=False)\nloader_b = DataLoader(train_set_b, batch_size=BATCH_SIZE, shuffle=False)'

In [13]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layers, out_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x, h = self.lstm(x.view(len(x),1,-1))
        x = self.fc1(x)
        return x

In [14]:
model_a = LSTM(SEQ_LEN, HIDDEN_DIM, N_LAYERS, OUT_DIM).cuda()
optimizer_a = optim.Adam(model_a.parameters(), lr=LR)

# model_b = LSTM(SEQ_LEN, HIDDEN_DIM, N_LAYERS, OUT_DIM).cuda()
# optimizer_b = optim.Adam(model_b.parameters(), lr=LR)

loss_fn = nn.MSELoss()

In [15]:
losses_a = []
losses_b = []

for epoch in range(EPOCHS):
    for i in tqdm(range(len(a)-SEQ_LEN)):
        Xa = a[i:i+SEQ_LEN]
        Ya = a[i+SEQ_LEN]

        Xa = Xa.view(-1, SEQ_LEN)

        out_a = model_a(Xa)
        loss_a = loss_fn(out_a, Ya)

        model_a.zero_grad()
        loss_a.backward()
        optimizer_a.step()
        losses_a.append(loss_a)

        
        '''Xb = b[i:i+SEQ_LEN]
        Yb = b[i+SEQ_LEN]

        Xb = Xb.view(-1, SEQ_LEN)


        out_b = model_b(Xb)
        loss_b = loss_fn(out_b, Yb)

        model_b.zero_grad()
        loss_b.backward()
        optimizer_b.step()
        losses_b.append(loss_b)'''

        

  0%|          | 1138/10568286 [00:43<119:15:48, 24.61it/s]